In [24]:
import pandas as pd
import numpy as np
import ast
import os

In [25]:
#reading tracks data
tracks = pd.read_csv("tracks.csv", index_col=0, header=[0, 1])

In [26]:
#dropping the extra columns
basic_columns = [('album','date_released'), ('album', 'title'), ('album', 'id'),('artist', 'id'), ('artist', 'name'),('track', 'genres_all'),('track', 'title'), ('track', 'duration'),('track', 'genre_top')]
tracks = tracks[basic_columns]
tracks.columns = tracks.columns.droplevel(0)
#Assigning new names to the column
tracks.columns = ['year_released', 'album', 'album_id', 'artist_id', 'artist',  'genres_all', 'track', 'duration','genre_top']


In [27]:
#checking the size of the dataset
tracks.shape

(106574, 9)

In [28]:
#Replace blank top genres with 'N/A'
tracks['genres_all'].fillna('N/A', inplace=True)
#Replace blank top genres with 'UNKNOWN'
tracks['genre_top'].fillna('UNKNOWN', inplace=True)
tracks.isnull().sum()

year_released    36280
album             1025
album_id             0
artist_id            0
artist               0
genres_all           0
track                1
duration             0
genre_top            0
dtype: int64

In [29]:
tracks.dtypes

year_released    object
album            object
album_id          int64
artist_id         int64
artist           object
genres_all       object
track            object
duration          int64
genre_top        object
dtype: object

In [30]:
# As there are many rows for which year_released is not avaialble. we replacing it with 0 value.
tracks['year_released'] = pd.DatetimeIndex(tracks['year_released']).year
tracks['year_released'].fillna(0, inplace=True)
tracks = tracks.astype({"year_released": int})

In [31]:
#Columns that should be categories
COLUMNS_cat = ['genre_top']
for column in COLUMNS_cat:
    tracks[column] = tracks[column].astype('category')

In [32]:
# converting to numeric value
COLUMNS_int = ['duration']
for column in COLUMNS_int:
    tracks[column] = pd.to_numeric(tracks[column], errors='coerce')
    tracks[column].fillna(0, inplace=True)

In [33]:
COLUMNS_list = ['genres_all']
#for column in COLUMNS_list:
#tracks[column] = tracks[column].values.tolist()
for column in COLUMNS_list:
    tracks[column] = tracks[column].map(ast.literal_eval)


In [34]:
tracks.head()

,year_released,album,album_id,artist_id,artist,genres_all,track,duration,genre_top
track_id,,,,,,,,,
2,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Food,168,Hip-Hop
3,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Electric Ave,237,Hip-Hop
5,2009,AWOL - A Way Of Life,1,1,AWOL,[21],This World,206,Hip-Hop
10,2008,Constant Hitmaker,6,6,Kurt Vile,[10],Freeway,161,Pop
20,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Spiritual Level,311,UNKNOWN


In [35]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Data columns (total 9 columns):
year_released    106574 non-null int32
album            105549 non-null object
album_id         106574 non-null int64
artist_id        106574 non-null int64
artist           106574 non-null object
genres_all       106574 non-null object
track            106573 non-null object
duration         106574 non-null int64
genre_top        106574 non-null category
dtypes: category(1), int32(1), int64(3), object(4)
memory usage: 7.0+ MB


In [36]:
unknown_genres = tracks.loc[(tracks['genre_top']=='UNKNOWN'), ['genre_top','genres_all']]
unknown_genres.head()

,genre_top,genres_all
track_id,,
20,UNKNOWN,"[17, 10, 76, 103]"
26,UNKNOWN,"[17, 10, 76, 103]"
30,UNKNOWN,"[17, 10, 76, 103]"
46,UNKNOWN,"[17, 10, 76, 103]"
48,UNKNOWN,"[17, 10, 76, 103]"


In [37]:
unknown_genres.shape

(56976, 2)

In [38]:
genres = pd.read_csv('genres.csv')
genres.head()

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5


In [39]:
top_genres = genres[genres.top_level==genres.genre_id]
top_genres.head()

,genre_id,#tracks,parent,title,top_level
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
7,8,868,0,Old-Time / Historic,8


In [40]:
genre_top_dict = dict(zip(top_genres.genre_id, top_genres.title))
print(genre_top_dict)

{2: 'International', 3: 'Blues', 4: 'Jazz', 5: 'Classical', 8: 'Old-Time / Historic', 9: 'Country', 10: 'Pop', 12: 'Rock', 13: 'Easy Listening', 14: 'Soul-RnB', 15: 'Electronic', 17: 'Folk', 20: 'Spoken', 21: 'Hip-Hop', 38: 'Experimental', 1235: 'Instrumental'}


In [41]:
for index, row in unknown_genres.iterrows():
    
#print(index,row)
    genres_top = row['genre_top']
    genres_all = row['genres_all']

    for genre in genres_all:
        if genre in list(genre_top_dict.keys()):
            tracks.loc[index, 'genre_top'] = genre_top_dict[genre]
            
tracks.head(10)

,year_released,album,album_id,artist_id,artist,genres_all,track,duration,genre_top
track_id,,,,,,,,,
2,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Food,168,Hip-Hop
3,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Electric Ave,237,Hip-Hop
5,2009,AWOL - A Way Of Life,1,1,AWOL,[21],This World,206,Hip-Hop
10,2008,Constant Hitmaker,6,6,Kurt Vile,[10],Freeway,161,Pop
20,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Spiritual Level,311,Pop
26,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Where is your Love?,181,Pop
30,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Too Happy,174,Pop
46,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Yosemite,104,Pop
48,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Light of Light,205,Pop


In [42]:
## Create a dictionary of genres genre_dict where key = genre_id : values = genre title

genre_all_dict = dict(zip(genres.genre_id, genres.title))
print(genre_all_dict)

{1: 'Avant-Garde', 2: 'International', 3: 'Blues', 4: 'Jazz', 5: 'Classical', 6: 'Novelty', 7: 'Comedy', 8: 'Old-Time / Historic', 9: 'Country', 10: 'Pop', 11: 'Disco', 12: 'Rock', 13: 'Easy Listening', 14: 'Soul-RnB', 15: 'Electronic', 16: 'Sound Effects', 17: 'Folk', 18: 'Soundtrack', 19: 'Funk', 20: 'Spoken', 21: 'Hip-Hop', 22: 'Audio Collage', 25: 'Punk', 26: 'Post-Rock', 27: 'Lo-Fi', 30: 'Field Recordings', 31: 'Metal', 32: 'Noise', 33: 'Psych-Folk', 36: 'Krautrock', 37: 'Jazz: Vocal', 38: 'Experimental', 41: 'Electroacoustic', 42: 'Ambient Electronic', 43: 'Radio Art', 45: 'Loud-Rock', 46: 'Latin America', 47: 'Drone', 49: 'Free-Folk', 53: 'Noise-Rock', 58: 'Psych-Rock', 63: 'Bluegrass', 64: 'Electro-Punk', 65: 'Radio', 66: 'Indie-Rock', 70: 'Industrial', 71: 'No Wave', 74: 'Free-Jazz', 76: 'Experimental Pop', 77: 'French', 79: 'Reggae - Dub', 81: 'Afrobeat', 83: 'Nerdcore', 85: 'Garage', 86: 'Indian', 88: 'New Wave', 89: 'Post-Punk', 90: 'Sludge', 92: 'African', 94: 'Freak-Folk'

In [43]:
def update_row(genres, genre_all_dict=genre_all_dict):
    row = []
    for genre_id in genres:
        row.append(genre_all_dict[genre_id])
    return row


tracks.rename({'genres_all' : 'genres_all_id'}, inplace=True, axis=1)
tracks['genres_all']= tracks['genres_all_id'].apply(update_row)
tracks.head(10)

,year_released,album,album_id,artist_id,artist,genres_all_id,track,duration,genre_top,genres_all
track_id,,,,,,,,,,
2,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Food,168,Hip-Hop,[Hip-Hop]
3,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Electric Ave,237,Hip-Hop,[Hip-Hop]
5,2009,AWOL - A Way Of Life,1,1,AWOL,[21],This World,206,Hip-Hop,[Hip-Hop]
10,2008,Constant Hitmaker,6,6,Kurt Vile,[10],Freeway,161,Pop,[Pop]
20,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Spiritual Level,311,Pop,"[Folk, Pop, Experimental Pop, Singer-Songwriter]"
26,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Where is your Love?,181,Pop,"[Folk, Pop, Experimental Pop, Singer-Songwriter]"
30,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Too Happy,174,Pop,"[Folk, Pop, Experimental Pop, Singer-Songwriter]"
46,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Yosemite,104,Pop,"[Folk, Pop, Experimental Pop, Singer-Songwriter]"
48,2009,Niris,4,4,Nicky Cook,"[17, 10, 76, 103]",Light of Light,205,Pop,"[Folk, Pop, Experimental Pop, Singer-Songwriter]"


In [44]:
filepath='tracks_info.csv'
tracks.to_csv(filepath)

In [45]:
filepath= 'echonest.csv'
echonest= pd.read_csv(filepath,index_col=0, header=[1, 2])

In [46]:
echonest.head(10)

audio_features                                                    \
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   
139            0.106550     0.260911  0.607067         0.835087  0.223676   
140            0.376312     0.734079  0.265685         0.669581  0.085995   
141            0.963657     0.435933  0.075632         0.345493  0.105686   
142            0.662881     0.379065  0.823856         0.910266  0.088705   
144            0.909011     0.443643  0.641997         0.924092  0.267669   

                                           metadata                     ...  \
         speechiness    tempo   valence  album_date         album_name  ...   
track_id                                                                ...   
2           0.159310  165.922  0.576661         NaN                NaN  ...   
3           0.461818  126.957  0.269240         NaN                NaN  ...   
5           0.124595  100.260  0.621661         NaN                NaN  ...   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker  ...   
134         0.525519  114.290  0.894072         NaN                NaN  ...   
139         0.030569  196.961  0.160267         NaN                NaN  ...   
140         0.039068  107.952  0.609991         NaN                NaN  ...   
141         0.026658   33.477  0.163950         NaN                NaN  ...   
142         0.079090  147.781  0.092868        2005     The Quiet Room  ...   
144         0.089659  128.537  0.788251         NaN                NaN  ...   

         temporal_features                                                    \
                       214        215       216       217       218      219   
track_id                                                                       
2                -1.992303   6.805694  0.233070  0.192880  0.027455  0.06408   
3                -1.582331   8.889308  0.258464  0.220905  0.081368  0.06413   
5                -2.288358  11.527109  0.256821  0.237820  0.060122  0.06014   
10               -3.662988  21.508228  0.283352  0.267070  0.125704  0.08082   
134              -1.452696   2.356398  0.234686  0.199550  0.149332  0.06440   
139              -3.078667  12.411567  0.270802  0.272700  0.025242  0.06404   
140              -0.934696  -0.260981  0.322232  0.277980  0.136747  0.07533   
141              -0.457298  -0.165962  0.437503  0.307390  0.368764  0.06263   
142              -1.125605   1.478418  0.215844  0.209660  0.015754  0.06921   
144              -3.454056  16.247305  0.245023  0.226080  0.033509  0.07066   

                                                     
               220       221        222         223  
track_id                                             
2          3.67696   3.61288  13.316690  262.929749  
3          6.08277   6.01864  16.673548  325.581085  
5          5.92649   5.86635  16.013849  356.755737  
10         8.41401   8.33319  21.317064  483.403809  
134       11.26707  11.20267  26.454180  751.147705  
139        2.43669   2.37265   3.897095   37.866043  
140        9.86272   9.78739  21.981621  562.229431  
141       11.18884  11.12621  13.481531  232.993546  
142        3.91102   3.84181  12.598523  346.813049  
144        2.98367   2.91301  10.123219  146.145737  

[10 rows x 249 columns]

In [47]:
echonest.info

<bound method DataFrame.info of          audio_features                                                    \
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476     1.062807e-02  0.177647   
3              0.374408     0.528643  0.817461     1.851103e-03  0.105880   
5              0.043567     0.745566  0.701470     6.967990e-04  0.373143   
10             0.951670     0.658179  0.924525     9.654270e-01  0.115474   
134            0.452217     0.513238  0.560410     1.944269e-02  0.096567   
139            0.106550     0.260911  0.607067     8.350870e-01  0.223676   
140            0.376312     0.734079  0.265685     6.695811e-01  0.085995   
141            0.963657     0.435933  0.075632     3.454935e-01  0.105686   
142            0.662881     0.379065  0.823856     9.102662e-01  0.088705   
144            0.909011     0.443643  0.6419

In [48]:
echonest.columns

MultiIndex(levels=[['audio_features', 'metadata', 'ranks', 'social_features', 'temporal_features'], ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '12

In [49]:
echonest_audio_features = echonest['audio_features']


In [50]:
echonest_audio_features.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [51]:
echonest_audio_features.describe()


,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
count,1.312900e+04,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000,13129.000000
mean,5.246876e-01,0.487290,0.537516,0.640536,0.187804,0.099174,123.080061,0.439761
std,3.837186e-01,0.190148,0.278049,0.361430,0.158051,0.137381,35.015137,0.276028
min,9.035000e-07,0.051307,0.000020,0.000000,0.025297,0.022324,12.753000,0.000010
25%,1.037726e-01,0.344759,0.321300,0.323466,0.101406,0.036932,95.967000,0.197321
50%,5.739848e-01,0.485635,0.549113,0.838134,0.119002,0.049019,120.057000,0.417743
75%,9.207270e-01,0.629094,0.776254,0.918244,0.211041,0.085452,145.318000,0.665575
max,9.957965e-01,0.968645,0.999964,0.998016,0.980330,0.966177,251.072000,0.999990


In [52]:
echonest_audio_features.to_csv('echonest_audio_features.csv')


In [53]:
filepath='echonest_audio_features.csv'
echonest_audio_features_copy = pd.read_csv(filepath, index_col='track_id')
echonest_audio_features_copy.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [54]:
tracks.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 106574 entries, 2 to 155320
Data columns (total 10 columns):
year_released    106574 non-null int32
album            105549 non-null object
album_id         106574 non-null int64
artist_id        106574 non-null int64
artist           106574 non-null object
genres_all_id    106574 non-null object
track            106573 non-null object
duration         106574 non-null int64
genre_top        106574 non-null category
genres_all       106574 non-null object
dtypes: category(1), int32(1), int64(3), object(5)
memory usage: 7.8+ MB


In [55]:
echonest_audio_features.head()


,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072


In [56]:
tracks = tracks.loc[tracks.index.isin(echonest_audio_features.index.values)]


In [57]:
tracks.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Data columns (total 10 columns):
year_released    13129 non-null int32
album            13129 non-null object
album_id         13129 non-null int64
artist_id        13129 non-null int64
artist           13129 non-null object
genres_all_id    13129 non-null object
track            13128 non-null object
duration         13129 non-null int64
genre_top        13129 non-null category
genres_all       13129 non-null object
dtypes: category(1), int32(1), int64(3), object(5)
memory usage: 988.0+ KB


In [58]:
tracks_labeled = echonest_audio_features.join(tracks, how='inner')
tracks_labeled.head()

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,year_released,album,album_id,artist_id,artist,genres_all_id,track,duration,genre_top,genres_all
track_id,,,,,,,,,,,,,,,,,,
2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Food,168,Hip-Hop,[Hip-Hop]
3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Electric Ave,237,Hip-Hop,[Hip-Hop]
5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,2009,AWOL - A Way Of Life,1,1,AWOL,[21],This World,206,Hip-Hop,[Hip-Hop]
10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008,Constant Hitmaker,6,6,Kurt Vile,[10],Freeway,161,Pop,[Pop]
134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,2009,AWOL - A Way Of Life,1,1,AWOL,[21],Street Music,207,Hip-Hop,[Hip-Hop]


In [59]:
tracks_labeled.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13129 entries, 2 to 124911
Data columns (total 18 columns):
acousticness        13129 non-null float64
danceability        13129 non-null float64
energy              13129 non-null float64
instrumentalness    13129 non-null float64
liveness            13129 non-null float64
speechiness         13129 non-null float64
tempo               13129 non-null float64
valence             13129 non-null float64
year_released       13129 non-null int32
album               13129 non-null object
album_id            13129 non-null int64
artist_id           13129 non-null int64
artist              13129 non-null object
genres_all_id       13129 non-null object
track               13128 non-null object
duration            13129 non-null int64
genre_top           13129 non-null category
genres_all          13129 non-null object
dtypes: category(1), float64(8), int32(1), int64(3), object(5)
memory usage: 2.4+ MB


In [60]:
filepath='tracks_cleaned.csv'
tracks_labeled.to_csv(filepath)